### Scraper for <a href="https://www.eiyokentei.com/category/column/"> Eiyokentei </a> website

In [7]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import os
import requests

In [8]:
def get_chrome_Driver():
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    return driver

def get_soup(URL):
    r = requests.get(URL)
    if r:
        driver = get_chrome_Driver()
        driver.get(URL)
        htmlContent = driver.page_source
        soup =  BeautifulSoup(htmlContent, 'html.parser')
        driver.close()
        return soup
    else:
        print("Error Fetching Soup, Scraping Ended")
        return False

def get_link_from_soup(soup):
    ul = soup.findAll('ul',{'class':'p-postList -type-card -pc-col3 -sp-col1'})[0]
    wanted_lis= ul.findAll('li',{'class':'p-postList__item'})
    link_list = []

    for li in wanted_lis:
        link = li.find('a',{'class':'p-postList__link'})
        link = str(link['href'])
        link_list.append(link)

    return link_list



def save_link_as_txt(link_list):
    file_path = "links/kaoruya.txt"
    # if file exist, get its content, merge to existing set and then overwrite
    if os.path.isfile(file_path):
        with open(file_path, 'r+', encoding="utf-8") as file:
            # get existing links in file
            old_links = set(file.readlines())
            # remove duplicates 
            link_list = link_list - old_links
            file.seek(0)
            for i in link_list:
                file.write(i+"\n")
            file.truncate()
    else: # if not, make new file
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        with open(file_path, "w", encoding="utf-8") as file:
            for i in link_list:
                file.write(i+"\n")
    print(f"Links Saved in {file_path}")
    return True

In [9]:
all_links = set()
for i in range(1,28):
    if i == 1:
        URL = 'https://kaoruya.org/blog/'
    else:
        URL = f"https://kaoruya.org/blog/page/{i}/"
    
    soup = get_soup(URL)
    if soup:
        current_links = get_link_from_soup(soup)
        if len(current_links) > 0: # continue until no tags are found 
            all_links.update(current_links)
        else: # if tags we want are not found
            break
    else: # if encountered HTML error such as 404
        break

save_link_as_txt(all_links)

Links Saved in links/kaoruya.txt


True